In [ ]:
import sys
sys.path.append("../")

import tensorflow as tf
import tensorflow_addons as tfa
from modules import utils, models
from tensorflow.keras import metrics
from tensorflow.keras import callbacks

# Training Single-Record Models:
## Metadata-Only Model:

In [ ]:
metadata_train_gen = utils.CombinedDataGen(data_file='../data/demo_train.csv',
                                           out_mode='meta',
                                           mode='train',
                                           shuffle=True,
                                           scaler_dir='../models/scalers')

metadata_valid_gen = utils.CombinedDataGen(data_file='../data/demo_valid.csv',
                                           out_mode='meta',
                                           mode='valid',
                                           shuffle=True,
                                           scaler_dir='../models/scalers')

train_ds = utils.get_ds_from_gen(metadata_train_gen,
                                 out_mode='meta',
                                 mode='train',
                                 batch_size=64,
                                 cache_dir='')
valid_ds = utils.get_ds_from_gen(metadata_valid_gen,
                                 out_mode='meta',
                                 mode='valid',
                                 batch_size=64,
                                 cache_dir='')

In [ ]:
metadata_mdl = models.init_meta_model()

metadata_mdl.compile(optimizer='adam', 
                     loss='binary_crossentropy',
                     metrics=[metrics.AUC()])

In [ ]:
es_cb = callbacks.EarlyStopping(monitor='val_loss',
                                min_delta=1e-8,
                                patience=10,
                                restore_best_weights=True)

metadata_mdl.fit(train_ds, 
                 epochs=10_000,
                 validation_data=valid_ds, 
                 callbacks=[es_cb])

## Image-Only Model:

In [ ]:
import sys
sys.path.append("../")

import tensorflow as tf
import tensorflow_addons as tfa
from modules import utils, models
from tensorflow.keras import metrics
from tensorflow.keras import callbacks

In [ ]:
image_train_gen = utils.CombinedDataGen(data_file='../data/demo_train.csv',
                                        out_mode='image',
                                        mode='train',
                                        shuffle=True,
                                        scaler_dir='../models/scalers')

image_valid_gen = utils.CombinedDataGen(data_file='../data/demo_valid.csv',
                                        out_mode='image',
                                        mode='valid',
                                        shuffle=True,
                                        scaler_dir='../models/scalers')

train_ds = utils.get_ds_from_gen(image_train_gen,
                                 out_mode='image',
                                 mode='train',
                                 batch_size=64,
                                 cache_dir='/home/yglaser/lus_scratch/tfdata_cache/im_train.tmp')
valid_ds = utils.get_ds_from_gen(image_valid_gen,
                                 out_mode='image',
                                 mode='valid',
                                 batch_size=64,
                                 cache_dir='/home/yglaser/lus_scratch/tfdata_cache/im_valid.tmp')

In [ ]:
image_mdl = models.init_image_model()

eps = 1_000
nb_steps = eps * (len(image_train_gen) // 64)

radam = tfa.optimizers.RectifiedAdam(lr=1e-3, 
                                     total_steps=nb_steps, 
                                     warmup_proportion=0.02, 
                                     min_lr=1e-5)
ranger = tfa.optimizers.Lookahead(radam, sync_period=7, slow_step_size=0.4)

image_mdl.compile(optimizer=ranger,
                  loss='binary_crossentropy',
                  metrics=[metrics.AUC()])

In [ ]:
es_cb = callbacks.EarlyStopping(monitor='val_loss',
                                min_delta=1e-8,
                                patience=10,
                                restore_best_weights=True)

image_mdl.fit(train_ds, 
              epochs=eps,
              validation_data=valid_ds, 
              callbacks=[es_cb],
              verbose=1)

In [ ]:
image_mdl.save('../models/image_single_rec.h5')

## Combined Model:

In [ ]:
import sys
sys.path.append("../")

import tensorflow as tf
import tensorflow_addons as tfa
from modules import utils, models
from tensorflow.keras import metrics
from tensorflow.keras import callbacks

In [ ]:
combined_train_gen = utils.CombinedDataGen(data_file='../data/demo_train.csv',
                                           out_mode='combined',
                                           mode='train',
                                           shuffle=True,
                                           scaler_dir='../models/scalers')

combined_valid_gen = utils.CombinedDataGen(data_file='../data/demo_valid.csv',
                                           out_mode='combined',
                                           mode='valid',
                                           shuffle=True,
                                           scaler_dir='../models/scalers')

train_ds = utils.get_ds_from_gen(combined_train_gen,
                                 out_mode='combined',
                                 mode='train',
                                 batch_size=64,
                                 cache_dir='')
valid_ds = utils.get_ds_from_gen(combined_train_gen,
                                 out_mode='combined',
                                 mode='valid',
                                 batch_size=64,
                                 cache_dir='')

In [ ]:
combined_mdl = models.init_combined_model(weights=['../models/image_single_rec.h5',
                                                   '../models/metadata_single_rec.h5'])

for l in combined_mdl.layers:
    assert l.trainable
    if l.name.startswith('img'):
        l.trainable = False
    if l.name.startswith('meta'):
        l.trainable = False
    if l.name.startswith('densenet'):
        for ll in l.layers:
            ll.trainable = False

combined_mdl.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=[metrics.AUC()])

In [ ]:
es_cb = callbacks.EarlyStopping(monitor='val_loss',
                                min_delta=1e-8,
                                patience=10,
                                restore_best_weights=True)

combined_mdl.fit(train_ds,
                 epochs=30,
                 validation_data=valid_ds, 
                 callbacks=[es_cb],
                 verbose=1)

In [ ]:
eps = 1_000
nb_steps = eps * (len(image_train_gen) // 64)

radam = tfa.optimizers.RectifiedAdam(lr=1e-3, 
                                     total_steps=nb_steps, 
                                     warmup_proportion=0.012, 
                                     min_lr=1e-5)
ranger = tfa.optimizers.Lookahead(radam, sync_period=7, slow_step_size=0.4)


for l in combined_mdl.layers:
    assert l.trainable
    if l.name.startswith('img'):
        l.trainable = False
    if l.name.startswith('meta'):
        l.trainable = False
    if l.name.startswith('densenet'):
        for ll in l.layers:
            ll.trainable = False
            
combined_mdl.compile(optimizer=ranger,
                     loss='binary_crossentropy',
                     metrics=[metrics.AUC()])

In [ ]:
combined_mdl.fit(train_ds,
                 epochs=eps,
                 validation_data=valid_ds, 
                 callbacks=[es_cb],
                 verbose=1)

In [ ]:
combined_mdl.save('../models/combined_single_rec.h5')